# TODO choose options

In [18]:
option = 'cge'
threshold = 5 # give integer as a percentage
project = 'Geothermal'
n_iter=100 # Number of MC iterations

# Files for simplified models coefficients
file_cge = 'Simplified models coefficients conventional - symbolic - thresholds.xlsx'
file_ege = 'Simplified models coefficients enhanced - symbolic - thresholds.xlsx'

# Set up geothermal

In [19]:
from setup_files_gsa import *

In [20]:
demand, methods, gt_model, parameters = setup_gt_project(project, option)

In [104]:
if option == 'cge':
    path_cge = os.path.join("generated_files", file_cge)
    coeffs = pd.read_excel(path_cge, sheet_name=['alpha_'+ str(threshold)+'%', 
                                                  'beta_'+ str(threshold)+'%'])
elif option == 'ege':
    path_ege = os.path.join("generated_files", file_ege)
    coeffs = pd.read_excel(path_cge, sheet_name=['chi_'+ str(threshold)+'%', 
                                                  'delta_'+ str(threshold)+'%'])

In [105]:
def simplified_model(coeffs, option, threshold):
    if option == 'cge':
        results = simplified_cge_model(coeffs, threshold)
    elif option == 'ege':
        results = simplified_ege_model(coeffs, threshold)
    return results

In [106]:
alpha_df = coeffs['alpha_' + str(threshold) + '%']
alpha_df = alpha_df.rename(columns={alpha_df.columns[0]: 'methods'})
beta_df  = coeffs['beta_'  + str(threshold) + '%']
beta_df  = beta_df.rename(columns={alpha_df.columns[0]: 'methods'})

In [107]:
alpha_df

,methods,alpha1,alpha2
0,"('ILCD 2.0 2018 midpoint no LT', 'climate chan...",1,0.00476


In [108]:
def simplified_cge_model(coeffs, option, threshold):
    '''
    alpha and beta are pandas DataFrame with methods as indices and coefficients as columns.
    ''' 
    alpha_df = coeffs['alpha_' + str(threshold) + '%']
    alpha_df = alpha_df.rename(columns={alpha_df.columns[0]: 'methods'})
    beta_df  = coeffs['beta_'  + str(threshold) + '%']
    beta_df  = beta_df.rename(columns={alpha_df.columns[0]: 'methods'})
    
    # Define methods
    ILCD_CC = ('ILCD 2.0 2018 midpoint no LT', 'climate change', 'climate change total')
    ILCD_EQ = [('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater and terrestrial acidification'),
               ('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater ecotoxicity'),
               ('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater eutrophication'),
               ('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'marine eutrophication'),
               ('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'terrestrial eutrophication')]
    ILCD_HH = [('ILCD 2.0 2018 midpoint no LT', 'human health', 'carcinogenic effects'),
               ('ILCD 2.0 2018 midpoint no LT', 'human health', 'ionising radiation'),
               ('ILCD 2.0 2018 midpoint no LT', 'human health', 'non-carcinogenic effects'),
               ('ILCD 2.0 2018 midpoint no LT', 'human health', 'ozone layer depletion'),
               ('ILCD 2.0 2018 midpoint no LT', 'human health', 'photochemical ozone creation'),
               ('ILCD 2.0 2018 midpoint no LT', 'human health', 'respiratory effects, inorganics')]
    ILCD_RE = [('ILCD 2.0 2018 midpoint no LT', 'resources', 'dissipated water'),
               ('ILCD 2.0 2018 midpoint no LT', 'resources', 'fossils'),
               ('ILCD 2.0 2018 midpoint no LT', 'resources', 'land use'),
               ('ILCD 2.0 2018 midpoint no LT', 'resources', 'minerals and metals')]
    
        
    # Calculate results
    results = {}   
    
    for method in methods:
        if method == ILCD_CC:
            results[method] = [ params["co2_emissions"][i] * alpha["alpha1"][str(method)] + alpha["alpha2"][str(method)]
                                for i in range(len_1) ]
        elif method in ILCD_EQ or method in ILCD_HH or method in ILCD_RE:
            results[method] = [ (( params["average_depth_of_wells"][i] * beta["beta1"][str(method)] + beta["beta2"][str(method)] )\
                               / params["gross_power_per_well"][i] ) + params["average_depth_of_wells"][i] * beta["beta3"][str(method)] + beta["beta4"][str(method)] 
                            for i in range(len_2) ]
        else:
            raise Exception("The simplified model does not work with the impact category:  ", method)
            
    return results    

In [ ]:
model_dict = {
    'cge': {
        '5%':
        '10%':
        '15%':
        '20%':
    },
    'ege':
}